<a href="https://colab.research.google.com/github/enesdemirag/gate-detection/blob/colab/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone git repository
!git clone https://github.com/enesdemirag/gate-detection.git

In [ ]:
# Install requirements
!pip3 install Cython
!pip3 install contextlib2
!pip3 install pillow
!pip3 install lxml
!pip3 install jupyter
!pip3 install matplotlib

In [ ]:
# Checkout simple-training branch
import os
os.chdir('gate-detection/')
!git checkout model

In [ ]:
# Clone object detection API
!git clone https://github.com/tensorflow/models.git

In [ ]:
# Install COCO API
!git clone https://github.com/cocodataset/cocoapi.git
os.chdir('cocoapi/PythonAPI/')
!make
!cp -r pycocotools ../../models/research/

In [ ]:
# Compile Protobuf
os.chdir('../../models/research/')
!protoc object_detection/protos/*.proto --python_out=.

# Add libraries to Python Path
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
# !set PYTHONPATH=/content/gate-detection/models/research : /content/gate-detection/models/research/slim

In [ ]:
# Install API
!sudo python3 setup.py install

In [ ]:
# Copy training files into API folder
os.chdir('../../')
!cp -r data/ models/research/object_detection/
!cp -r images/ models/research/object_detection/
!cp -r model/ models/research/object_detection/
!cp -r training/ models/research/object_detection/

In [ ]:
# Get inside object_detection folder
os.chdir('models/research/object_detection/')

In [ ]:
# Run training script
!python3 legacy/train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/ssd_mobilenet.config